In [2]:
# Install and import packages
!pip install pandas openpyxl matplotlib plotly -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
print("Setup complete!")

Setup complete!


In [3]:
# Upload your categorized Excel file
print("Please upload: categorized_orders_clean.xlsx")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"\nUploaded: {filename}")

Please upload: categorized_orders_clean.xlsx


Saving categorized_orders_clean.xlsx to categorized_orders_clean.xlsx

Uploaded: categorized_orders_clean.xlsx


In [5]:
# Load data
df = pd.read_excel(filename)
print(f"Loaded {len(df)} rows")
print(f"Columns: {df.columns.tolist()}")

# Get order column (first column)
order_col = df.columns[0]

# Filter to parent rows only (rows with order number)
parent_df = df[df[order_col].notna()].copy()
print(f"\n{len(parent_df)} unique orders")

Loaded 3086 rows
Columns: ['oa', 'Outlet', 'Order Type', 'Order Date', 'Contact no', 'Amount', 'Reason', 'Remark', 'Void By ', 'Void Date', 'Placed By', 'Order Time', 'Item', 'Qty', 'Rate', 'Predicted_Category', 'Extracted_New_Bill']

824 unique orders


In [6]:
# Get month from data for report title
if 'Order Date' in parent_df.columns:
    try:
        dates = pd.to_datetime(parent_df['Order Date'], errors='coerce')
        if dates.notna().any():
            month_str = dates.dropna().iloc[0].strftime('%B, %Y')
        else:
            month_str = "Current Period"
    except:
        month_str = "Current Period"
else:
    month_str = "Current Period"

display(HTML(f'''
<div style="text-align:center; padding:20px; background:#1976D2; color:white; border-radius:10px;">
    <h1>VOID BILLS ANALYSIS</h1>
    <h3>Month of {month_str}</h3>
</div>
'''))

---
## 1. Findings
### 1.1 Identification of Special Incidents

In [8]:
# Key statistics
total_voids = len(parent_df)

# Top outlet
outlet_counts = parent_df['Outlet'].value_counts()
top_outlet = outlet_counts.index[0]
top_outlet_count = outlet_counts.iloc[0]

# Main reason
reason_counts = parent_df['Predicted_Category'].value_counts()
main_reason = reason_counts.index[0]
main_reason_count = reason_counts.iloc[0]

# Order types
order_types = parent_df['Order Type'].dropna().unique()[:4]
order_types_str = ", ".join(order_types)

display(HTML(f'''
<div style="padding:15px; border-left:4px solid #1976D2; margin:10px 0;">
    <h4>Key Findings:</h4>
    <ol>
        <li>The total number of void ({order_types_str}) bills was <b>{total_voids}</b>.</li>
        <li>The <b>{top_outlet}</b> outlet had the highest void bills. It was <b>{top_outlet_count}</b>.</li>
        <li>The main reason for the highest number of bill voided was <b>{main_reason}</b>. It was <b>{main_reason_count}</b>.</li>
    </ol>
</div>
'''))

### 1.2 Void Bills with Significant Total Amount

In [9]:
# Top void bills by amount
cols = [order_col, 'Outlet', 'Order Type', 'Order Date', 'Reason', 'Void By ', 'Amount']
available_cols = [c for c in cols if c in parent_df.columns]

top_amount = parent_df.nlargest(10, 'Amount')[available_cols].copy()
top_amount['Amount'] = top_amount['Amount'].apply(lambda x: f"{x:,.2f}")

display(HTML("<h4>Top 10 Void Bills by Amount:</h4>"))
display(top_amount.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,oa,Outlet,Order Type,Order Date,Reason,Void By,Amount
2218,W72045,Wattala,Take Away,2025-10-11 00:00:00,Product Testing,24632.000000,"126,752.21"
1893,G00265,One Galle Face,Take Away,2025-10-11 00:00:00,cux want to change 25 seafood melts to 25 meatfieast melts.new bill no G00280.MOD AVISHKA cashier yashara.time 12.19pm,29082.000000,"48,600.08"
2093,J48175,Rajagiriya,Take Away,2025-10-04 00:00:00,Customer want to have a Promotion,30137.000000,"46,200.08"
65,HM8435,Anuradhapura,Delivery,2025-07-19 00:00:00,wrong orders,23266.000000,"40,656.50"
248,HM9137,Anuradhapura,Delivery,2025-07-19 00:00:00,wrong orders,35972.000000,"40,656.50"
2320,N47044,Nuwara Eliya,Take Away,2025-07-05 00:00:00,Customer wanted to change the order,9506.000000,"32,740.05"
2351,N48431,Nuwara Eliya,Dine In,2025-07-15 00:00:00,Customer wanted to change the order,9506.000000,"22,876.00"
2862,D22897,Thimbirigasyaya,Take Away,2025-07-22 00:00:00,Customer wanted to change the order,5844.000000,"22,525.04"
2528,P48517,Peradeniya,Dine In,2025-07-23 00:00:00,Customer wanted to change the order,2704.000000,"21,897.75"
1385,O41441,Kothalawala,Take Away,2025-10-05 00:00:00,Customer want to have a Promotion,28790.000000,"17,700.03"


---
## 2. Facts
### 2.1 Channel-wise Number of Void Bills

In [10]:
# Channel-wise pivot table
channel_pivot = pd.crosstab(parent_df['Outlet'], parent_df['Order Type'], margins=True, margins_name='Total')
channel_pivot = channel_pivot.sort_values('Total', ascending=False)

display(HTML("<h4>Channel-wise Void Bills by Outlet:</h4>"))
display(channel_pivot.head(20).style.background_gradient(cmap='Blues').set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

Order Type,Aggregator,Delivery,Dine In,Take Away,Total
Outlet,,,,,
Total,169,306,159,190,824
Negambo,32,6,1,3,42
Wattala,5,11,5,11,32
Kochchikade,27,1,2,2,32
Rajagiriya,1,16,8,4,29
Union Place,1,15,0,5,21
Trincomalee,0,2,7,12,21
Bambalapitiya,5,11,0,2,18
Katugasthota,13,1,2,2,18


In [11]:
# Channel-wise bar chart
chart_data = channel_pivot.drop('Total', errors='ignore').drop('Total', axis=1, errors='ignore').head(15)

fig = go.Figure()
colors = {'Aggregator': '#4CAF50', 'Delivery': '#2196F3', 'Dine In': '#FFC107', 'Take Away': '#9C27B0'}

for col in chart_data.columns:
    fig.add_trace(go.Bar(name=col, x=chart_data.index, y=chart_data[col], marker_color=colors.get(col, '#607D8B')))

fig.update_layout(title='Channel-wise Void Bills', barmode='group', xaxis_tickangle=-45, height=500,
                  legend=dict(orientation="h", yanchor="bottom", y=1.02))
fig.show()

### 2.2 Outlets-wise Highest Total Void Bill Value

In [12]:
# Value by outlet and order type
value_pivot = parent_df.pivot_table(values='Amount', index='Outlet', columns='Order Type', aggfunc='sum', fill_value=0)
value_pivot['Total (Rs.)'] = value_pivot.sum(axis=1)
value_pivot = value_pivot.sort_values('Total (Rs.)', ascending=False)

# Format for display
value_display = value_pivot.copy()
for col in value_display.columns:
    value_display[col] = value_display[col].apply(lambda x: f"{x:,.2f}")

display(HTML("<h4>Outlet-wise Total Void Bill Value:</h4>"))
display(value_display.head(15).style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

Order Type,Aggregator,Delivery,Dine In,Take Away,Total (Rs.)
Outlet,,,,,
Wattala,"10,773.87","26,505.99","29,347.50","156,912.26","223,539.62"
Rajagiriya,"1,856.25","54,722.43","32,958.43","54,870.09","144,407.19"
Anuradhapura,0.00,"97,394.46","26,028.44","16,630.03","140,052.93"
Nuwara Eliya,0.00,"2,980.00","50,342.25","58,650.10","111,972.35"
Union Place,696.20,"70,513.38",0.00,"20,800.03","92,009.61"
Negambo,"58,921.40","20,321.03","2,515.50","8,380.01","90,137.94"
One Galle Face,"3,286.31",0.00,"32,373.09","48,600.08","84,259.47"
Kochchikade,"57,622.45","1,590.00","8,277.50","14,620.02","82,109.97"
Trincomalee,0.00,"5,276.75","33,378.48","43,042.07","81,697.30"


In [13]:
# Pie chart
top_outlets = value_pivot['Total (Rs.)'].head(10)
fig = px.pie(values=top_outlets.values, names=top_outlets.index, title='TOTAL VOID BILL VALUE (Top 10 Outlets)', hole=0.3)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

### 2.3 Summarize the Reasons for Void Bills

In [15]:
# Top 2 reasons
top_2_reasons = reason_counts.head(5).reset_index()
top_2_reasons.columns = ['Void Reason', 'Number of Void']

display(HTML("<h4>Most Common Reasons:</h4>"))
display(top_2_reasons.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,Void Reason,Number of Void
0,cus. Change the order,171
1,promotion,130
2,Customer denied the order,115
3,Cashier mistake,70
4,order cancelled by aggregator,53


### 2.4 Bills Voided Due to Customer Requests for Changes

In [16]:
# Customer change orders
change_orders = parent_df[parent_df['Predicted_Category'] == 'cus. Change the order'].sort_values('Amount', ascending=False)

display_cols = [order_col, 'Outlet', 'Order Type', 'Order Date', 'Reason', 'Void By ', 'Amount']
avail_cols = [c for c in display_cols if c in change_orders.columns]

change_display = change_orders[avail_cols].head(20).copy()
if 'Amount' in change_display.columns:
    change_display['Amount'] = change_display['Amount'].apply(lambda x: f"{x:,.2f}")

display(HTML(f"<h4>Bills Voided for Customer Order Changes ({len(change_orders)} orders):</h4>"))
display(change_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,oa,Outlet,Order Type,Order Date,Reason,Void By,Amount
1893,G00265,One Galle Face,Take Away,2025-10-11 00:00:00,cux want to change 25 seafood melts to 25 meatfieast melts.new bill no G00280.MOD AVISHKA cashier yashara.time 12.19pm,29082.000000,"48,600.08"
556,N27164,Bandaragama,Dine In,2025-07-10 00:00:00,Customer wanted to change the order,27756.000000,"15,974.50"
1649,S35048,Matugama,Dine In,2025-10-08 00:00:00,Customer wanted to change the order,21319.000000,"14,157.75"
1524,G52009,Kurunegala 3,Take Away,2025-10-18 00:00:00,Customer wanted to change the order,24826.000000,"12,180.02"
750,JO6506,Borella,Delivery,2025-10-05 00:00:00,cx wants to at 7.30,23266.000000,"10,879.54"
1086,W74129,Kandy,Take Away,2025-10-13 00:00:00,Customer wanted to change the order,21870.000000,"10,870.02"
929,A06659,Hatton,Take Away,2025-10-21 00:00:00,Customer wanted to change the order,11266.000000,"10,240.02"
2556,P25214,Pilimathalawa,Take Away,2025-07-06 00:00:00,Customer wanted to change the order,31123.000000,"10,210.02"
2956,W22937,Wadduwa,Take Away,2025-07-21 00:00:00,Customer wanted to change the order,3609.000000,"8,820.01"
1389,O42463,Kothalawala,Dine In,2025-10-18 00:00:00,Customer wanted to change the order,5382.000000,"8,686.00"


### 2.5 Bills Voided for Customer Cancel Order

In [17]:
# Customer cancel orders
cancel_orders = parent_df[parent_df['Predicted_Category'] == 'Customer Cancel order'].sort_values('Amount', ascending=False)

cancel_display = cancel_orders[avail_cols].head(25).copy()
if 'Amount' in cancel_display.columns:
    cancel_display['Amount'] = cancel_display['Amount'].apply(lambda x: f"{x:,.2f}")

display(HTML(f"<h4>Bills Voided for Customer Cancel Order ({len(cancel_orders)} orders):</h4>"))
display(cancel_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))

,oa,Outlet,Order Type,Order Date,Reason,Void By,Amount
2676,HR7473,Pita Kotte,Delivery,2025-07-25 00:00:00,customer want to cancel the order,6873.000000,"9,675.00"
1020,JX8240,Kadawatha,Delivery,2025-10-19 00:00:00,cx wanted to cancel,23266.000000,"7,267.00"
2205,JR6143,Union Place,Delivery,2025-10-10 00:00:00,customer want to cancel the order,24970.000000,"6,426.00"
1047,JW2879,Kaduwela,Delivery,2025-10-17 00:00:00,"Customer want to cancel ,",23266.000000,"5,439.50"
1072,JX8807,Kandana,Delivery,2025-10-19 00:00:00,cu wont to cancel,10971.000000,"4,912.75"
2625,HB7820,Pita Kotte,Delivery,2025-07-02 00:00:00,customr cancal,22798.000000,"4,770.00"
1552,JY7227,Maharagama,Delivery,2025-10-20 00:00:00,customer cancel order n.b 08 correct order n.b-093,23266.000000,"4,536.50"
2377,G77682,One Galle Face,Dine In,2025-07-12 00:00:00,"cux canceled the order after they didnt had a table to eat.cashier nadunka mod avishka/partheeban time the customer went at 2.20pm.evidance oshen,dileepa,nethmi",29082.000000,"4,525.75"
708,JO4098,Badulla,Delivery,2025-10-05 00:00:00,customer cancel the order customer (customer not place the order),34652.000000,"4,429.00"
2784,HU6340,Rajagiriya,Delivery,2025-07-29 00:00:00,cncl,23266.000000,"3,837.75"


### 2.6 The Reasons for All Void Bills

In [21]:
# All reasons breakdown
all_reasons = reason_counts.reset_index()
all_reasons.columns = ['Void Reason', 'Number Of Void']

# Friendly names mapping
name_map = {
    'cus. Change the order': 'Change of customer request',
    'promotion': 'Promotion',
    'Cashier mistake': 'Cashier mistake',
    'Customer denied the order': 'Customer denied the order',
    'Customer Cancel order': 'Customer Cancel order',
    'cus.related issue': 'Customer Related Issue',
    'grid issue': 'Grid issue',
    'phone': 'Contact Number Issues',
    'order without reason/ remark': 'Orders Without Reason / Remark',
    'Order delay': 'Order Delay',
    'order type change': 'Order Type Change',
    'system issue': 'System issue / breakdown',
    'rider issue': 'Riders related issues',
    'double punch': 'Order Double Punched',
    'payment issue': 'Payment Issues',
    'out of stock': 'Out of Stock',
    'location': 'Location',
    'testing': 'Testing',
    'Call Center mistake': 'CSR Issue',
    'product issue or complain': 'Product issue or complain',
    'order cancelled by aggregator': 'Order cancelled by aggregator',
    'other': 'Other'
}

all_reasons['Void Reason'] = all_reasons['Void Reason'].map(name_map).fillna(all_reasons['Void Reason'])

display(HTML("<h4>Complete Breakdown of Void Reasons:</h4>"))
display(all_reasons.style.set_properties(**{'background-color': ''}).set_table_styles([{'selector': 'th', 'props': [('background-color', '#FF8F00')]}]))

,Void Reason,Number Of Void
0,Change of customer request,171
1,Promotion,130
2,Customer denied the order,115
3,Cashier mistake,70
4,Order cancelled by aggregator,53
5,Contact Number Issues,41
6,Location,37
7,Customer Cancel order,35
8,CSR Issue,28
9,Riders related issues,22


In [22]:
# Bar chart for all reasons
fig = px.bar(all_reasons, x='Void Reason', y='Number Of Void', title='Void Bills by Reason Category',
             color='Number Of Void', color_continuous_scale='Blues')
fig.update_layout(xaxis_tickangle=-45, height=500, showlegend=False)
fig.show()

### 2.7 Delayed Void Processing Time

In [25]:
# Time gap analysis
if 'Order Time' in parent_df.columns and 'Void Date' in parent_df.columns:
    delayed = parent_df.copy()
    try:
        delayed['Order_DT'] = pd.to_datetime(delayed['Order Time'], errors='coerce')
        delayed['Void_DT'] = pd.to_datetime(delayed['Void Date'], errors='coerce')
        delayed['Time_Gap_Hours'] = (delayed['Void_DT'] - delayed['Order_DT']).dt.total_seconds() / 3600

        # Filter significant delays (> 2 hours)
        delayed_orders = delayed[delayed['Time_Gap_Hours'] > 2].sort_values('Time_Gap_Hours', ascending=False)

        if len(delayed_orders) > 0:
            delay_cols = [order_col, 'Outlet', 'Reason', 'Order_DT', 'Void_DT', 'Time_Gap_Hours']
            delay_display = delayed_orders[[c for c in delay_cols if c in delayed_orders.columns]].head(15).copy()
            delay_display['Time_Gap_Hours'] = delay_display['Time_Gap_Hours'].apply(lambda x: f"{int(x)}:{int((x%1)*60):02d}")

            display(HTML(f"<h4>Orders with Delayed Void Processing ({len(delayed_orders)} orders >2hr):</h4>"))
            display(delay_display.style.set_table_styles([{'selector': 'th', 'props': [('background-color', '#1976D2'), ('color', 'white')]}]))
        else:
            print("No significantly delayed orders found.")
    except Exception as e:
        print(f"Could not calculate time gaps: {e}")
else:
    print("Time columns not available for delay analysis.")

,oa,Outlet,Reason,Order_DT,Void_DT,Time_Gap_Hours
2693,HH7716,Ragama,security department not entered,2025-07-11 18:43:22,2025-07-12 10:15:34,15:32
538,HP7723,Bambalapitiya,nan,2025-07-22 20:29:32,2025-07-23 08:50:04,12:20
1027,K27443,Kaduwela,Customer denied the order,2025-10-05 11:15:00,2025-10-05 23:07:48,11:52
530,HL0949,Bambalapitiya,nan,2025-07-15 22:50:11,2025-07-16 10:37:34,11:47
2153,JR0303,Southern Express Colombo,Customer Due To Direction Delay He Denied The Order,2025-10-09 11:00:03,2025-10-09 22:39:36,11:39
2711,HI1372,Ragama,nan,2025-07-11 22:43:01,2025-07-12 10:15:29,11:32
1470,A16651,Kurana,Customer denied the order,2025-10-04 12:03:00,2025-10-04 23:08:15,11:05
2298,Y51339,Nugegoda,Customer wanted to change the order,2025-07-09 12:32:00,2025-07-09 23:35:59,11:03
2501,HQ4852,Pelawatta,The customer wanted to cancel the order. New bill 07,2025-07-24 11:47:02,2025-07-24 22:37:51,10:50
2275,Z61051,Negambo,Customer denied the order,2025-07-28 12:53:00,2025-07-28 23:40:14,10:47


---
## Summary

In [31]:
# Final summary
total_value = parent_df['Amount'].sum()

display(HTML(f'''
<div style="padding:20px; border-radius:10px; color:white;">
    <h2>Report Summary</h2>
    <table style="width:100%; color:white; font-size:16px;">
        <tr><td><b>Total Void Bills:</b></td><td>{total_voids}</td></tr>
        <tr><td><b>Total Void Value:</b></td><td>Rs. {total_value:,.2f}</td></tr>
        <tr><td><b>Top Outlet:</b></td><td>{top_outlet} ({top_outlet_count} voids)</td></tr>
        <tr><td><b>Main Reason:</b></td><td>{main_reason} ({main_reason_count} cases)</td></tr>
        <tr><td><b>Number of Outlets:</b></td><td>{parent_df['Outlet'].nunique()}</td></tr>
    </table>
</div>
'''))

Total Void Bills:,824
Total Void Value:,"Rs. 3,268,438.39"
Top Outlet:,Negambo (42 voids)
Main Reason:,cus. Change the order (171 cases)
Number of Outlets:,111


## Export Results

In [29]:
# Export to Excel
output_file = 'Void_Bills_Report_Output.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    parent_df.to_excel(writer, sheet_name='All Orders', index=False)
    all_reasons.to_excel(writer, sheet_name='Reason Summary', index=False)
    channel_pivot.to_excel(writer, sheet_name='Channel-wise')
    value_pivot.to_excel(writer, sheet_name='Outlet Values')

print(f"Exported to: {output_file}")
files.download(output_file)

Exported to: Void_Bills_Report_Output.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
### Report Complete!